<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/Bestvevery_word_result_rag_wiki_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wikipedia
!pip install faiss-cpu
!pip install sentence-transformers

In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [ ]:
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 63.0 MB/s eta 0:00:00


In [24]:
import wikipedia

import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


In [27]:
#pip install -U sentence-transformers


# 1. Fetch Documents from Wikipedia

def fetch_wikipedia_data(query, num_results=5):
    """Fetches Wikipedia articles related to a query."""
    try:
        search_results = wikipedia.search(query, results=num_results)
        documents = []
        for title in search_results:
            try:  # Handle potential DisambiguationError or PageError
                page = wikipedia.page(title)
                documents.append(page.content)
            except (wikipedia.exceptions.DisambiguationError, wikipedia.exceptions.PageError):
                print(f"Skipping page '{title}' due to error.")  # Handle errors gracefully
        return documents
    except Exception as e: # Catch other potential errors
        print(f"Error fetching Wikipedia data: {e}")
        return []



# 2. Sentence Splitting and Embedding Generation

model = SentenceTransformer('all-mpnet-base-v2')  # Or any other suitable model

def embed_sentences(documents):
    sentences = []
    for doc in documents:
        for sentence in doc.split('. '):  # Simple sentence splitting, consider more robust methods
            sentences.append(sentence.strip())  # Remove leading/trailing whitespace
    embeddings = model.encode(sentences)
    return sentences, embeddings

# 3. FAISS Index Creation and Search

def search_similar_sentences(query, sentences, embeddings, top_k=10):
    query_embedding = model.encode([query])[0]

    index = faiss.IndexFlatL2(embeddings.shape[1])  # Build L2 index
    index.add(embeddings)

    D, I = index.search(np.array([query_embedding]), k=top_k)  # Search
    similar_sentences = [sentences[i] for i in I[0]]
    similarity_scores = D[0]
    return similar_sentences, similarity_scores



# 4. Main Execution

def rag_pipeline(query, num_wikipedia_results=5, top_k_sentences=10):

    documents = fetch_wikipedia_data(query, num_results=num_wikipedia_results)
    if not documents:  # Check if documents were successfully retrieved
        print("No Wikipedia documents found for the query.")
        return

    sentences, embeddings = embed_sentences(documents)
    similar_sentences, scores = search_similar_sentences(query, sentences, embeddings, top_k=top_k_sentences)

    print("Query:", query)
    print("\nTop Similar Sentences:")
    for sentence, score in zip(similar_sentences, scores):
        print(f"Score: {score:.4f}: {sentence}")

  #  print("\nAll Sentences from Wikipedia Dataset:")
    #for sentence in sentences:
      #  print(sentence)



# Example Usage:

user_query = "Amitabh Bachan superhit films and awards"
rag_pipeline(user_query)

Query: Amitabh Bachan superhit films and awards

Top Similar Sentences:
Score: 0.4964: Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born Amitabh Srivastava; 11 October 1942) is an Indian actor who works in Hindi cinema
Score: 0.5338: The following is the listing of books focused on his life career:

Amitabh Bachchan: the Legend was published in 1999,
To be or not to be: Amitabh Bachchan in 2004,
AB: The Legend (A Photographer's Tribute) in 2006,
Amitabh Bachchan: Ek Jeevit Kimvadanti in 2006,
Amitabh: The Making of a Superstar in 2006,
Looking for the Big B: Bollywood, Bachchan and Me in 2007 and
Bachchanalia in 2009.


== Awards and honours ==

Apart from industry awards won for his performances throughout the years, Bachchan has received several honours for his achievements in the Indian film industry
Score: 0.6248: He is a recipient of several accolades including six National Film Awards and sixteen Filmfare Awards.
Amitabh Bachchan was born in 1942 in Allahabad (now Prayagr